<a href="https://colab.research.google.com/github/cnai-lab/qpsychometric/blob/main/validations/ambivalent_sexism_inventory/publish_experiments_ASI_EN_NSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import torch
import pandas as pd
from pathlib import Path
import gc
from tqdm.auto import tqdm
import warnings
import pingouin as pg
from sentence_transformers import SentenceTransformer, util

device = 0 if torch.cuda.is_available() else -1
print(device)

0


In [ ]:
!pip install qlatent

In [2]:
from qlatent.qnsp.qnsp import * 
from qlatent.qnsp.qnsp import QNSP

2024-10-08 10:37:44.606170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 10:37:44.621226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 10:37:44.625767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 10:37:44.636882: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-08 10:37:46.152914: W tensorflow/compiler/tf2

In [3]:
softmax_files = [True, False]

def split_question(Q, index, scales, softmax, filters):
  result = []
  for s in scales:
    q = QCACHE(Q(index=index, scale=s))
    for sf in softmax:
      for f in filters:
        if sf:            
            qsf = QSOFTMAX(q,dim=[index[0], s])
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print((index, s),sf,f)
            result.append(qsf_f)
        else:
            qsf = QPASS(q,descupdate={'softmax':''})
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print(s,sf,f)
            result.append(qsf_f)
  return result

def dict_pos_neg(pos, neg, w):
  return dict(dict_same_weight(1.0*w/len(pos),pos), **dict_same_weight(-1.0*w/len(neg),neg))

def print_permutations(q):
#     for q in Q1s:
    W = q._pdf['W']
    print(q._descriptor)
    for i, (kmap, w) in enumerate(zip(q._keywords_map, W)):
        context = q._context_template.format_map(kmap)
        answer = q._answer_template.format_map(kmap)
#         sexisem_score = sexisem_classifier(context.strip('.') + ' ' +answer)
        print(f'{i}.',context ,'->', answer, w)
#     break


frequency_weights:SCALE = {
    'never':-4,
    'very rarely':-3,
    'seldom':-2,
    'rarely':-2,
    'frequently':2,
    'often':2,
    'very frequently':3,
    'always':4,    
}
    
agree_weights:SCALE = {
    'strongly disagree':-4,
    'disagree':-3,
    'somewhat disagree':-2,
    'neutral':1,
    'somewhat agree':2,
    'agree':3,
    'strongly agree':4,    
}
    
intensifiers_fraction_without_none:SCALE={
            "few":1,
            "some":2,
            "many":3,
            "most":4,
            "all":5,
        }
    

# Load Models

In [4]:
nsp = NextSentencePredictionPipeline('google-bert/bert-base-uncased', device=device)
nsp.model_identifier = 'google-bert/bert-base-uncased'

/home/shistikk/.conda/envs/myenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
gc.collect()
torch.cuda.empty_cache()

# Linguastic acceptability

In [6]:
sentence_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
cola = pipeline("text-classification","mrm8488/deberta-v3-small-finetuned-cola", device=device)

import os
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu

def linguistic_acceptabilities(q, index, scale,question_name, student_id, output_path=Path(''), save_to_file=False):
    score_by_cola_lst=[]
    score_of_semantic_distance_lst=[]
    score_by_bleu_lst=[]
    kmap_lst=[]
    question_name_lst=[]
    description = q._descriptor
    strFactor=description['Factor']
    strOrdinal=str(description.get('Ordinal', 0))
    ##cleaning the string to get the original question
    strOriginal= description['Original']
    strOriginal = 'none' if strOriginal is None else strOriginal
    strOriginal=strOriginal.replace(strFactor,'',1)
    strOriginal=strOriginal.replace(strOrdinal,'',1)
    strOriginal=strOriginal.replace('.','',1)
    strOriginal=strOriginal.strip() #the original question
    rows = []
    
    partial_internal_consistency = partial(q.internal_consistency, filter={}, index=index , scale=scale)
    try:
        silhouette_score = partial_internal_consistency(measure='silhouette_score', metric='correlation')
    except Exception as e:
        print(e)
        print('silhouette_score is set to -1')
        silhouette_score = -1
        
    if hasattr(q, 'linguistic_acceptability'):
        q.linguistic_acceptability['silhouette_score'] = silhouette_score
        return q.linguistic_acceptability

    for kmap in q._keywords_map:
        score = {}
        score['question_name'] = question_name
        context = q._prompt.format_map(kmap)
        answer = q._next_sentence.format_map(kmap)
        score['original_question'] = strOriginal


        cola_score = cola(context +" "+ answer)[0].get('score')
        score['cola_score'] = cola_score
        score['param'] = kmap
        strPermutation= context +" "+ answer
        # sentences = [context +" "+ answer]
        score['question_permutation'] = strPermutation
        #Compute embedding for both lists
        embeddings1 = sentence_embedding_model.encode(strOriginal, convert_to_tensor=True)
        embeddings2 = sentence_embedding_model.encode(strPermutation, convert_to_tensor=True)

        #Compute cosine-similarities
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        score['semantic_similarity'] = cosine_scores.item()

        score['silhouette_score'] = silhouette_score
        rows.append(score)


    filename = output_path / 'linguistic_acceptabilities.csv'
    df = pd.DataFrame(rows)
    df['student_id'] = student_id
    df = df[['student_id', 'question_name','original_question', 'param','question_permutation','cola_score','semantic_similarity','silhouette_score']]
    if save_to_file:
        if filename.exists():
            df.to_csv(filename, index=False, header=None, mode='a', encoding='utf-8-sig')
        else:
            df.to_csv(filename, index=False, encoding='utf-8-sig')
#     print(f"Linguistic acceptabilities saved in {filename}")
    q.linguistic_acceptability = df
    return df

/home/shistikk/.conda/envs/myenv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/shistikk/.conda/envs/myenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/shistikk/.conda/envs/myenv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to 

# ASI

## Hostile sexism

### H 2. Many women are actually seeking special favors, such as hiring policies that favor them over men

In [6]:
kw_fairness = ["equal chances", 'fair opportunities']
kw_favors   = ['special favors', 'preferential treatment']
dict_objective = dict_pos_neg(kw_favors, kw_fairness,1)

class ASIQ2(QNSP):
  """
  H 2. Many women are actually seeking special favors, such as hiring policies that favor them over men,
  under the guise of asking for "equality."

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Women {frequency} ask for equality.",
        next_sentence = "Women are actually seeking {index}.",
        dimensions={
                    'frequency': frequency_weights,  
                    'index': dict_objective,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":2,
                      "Original":'H 2. Many women are actually seeking special favors, such as hiring policies that favor them over men, under the guise of asking for "equality."'
        },
        **kwargs,
    )
    
Q2s = split_question(ASIQ2,
                      index=["index",],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ2().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q2s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.39031386375427246
Mean score unfiltered [-2.0..2.0]: 3.4924596548080444e-09
Internal consistency (silhouette, correlation) for unfiltered: 0.0926709920167923
Internal consistency (Calinski&Harabasz)  for unfiltered: 1.7894736842105263
Internal consistency (Davies&Bouldin) for unfiltered: 1.252966855721656


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]


/home/shistikk/.conda/envs/myenv/lib/python3.11/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7fe6041ba8e0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
equal chances,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
fair opportunities,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
preferential treatment,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
special favors,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'index', 'frequency'} {'frequency'} {'index'}
[]
['equal chances', 'fair opportunities'] Alpha: 0.6436781609195403
['special favors', 'preferential treatment'] Alpha: 0.28125
Global alpha: 0.42105263157894746


/home/shistikk/.conda/envs/myenv/lib/python3.11/site-packages/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7fe6041ba8e0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### H 4. Most women interpret innocent remarks or acts as being sexist.

In [10]:
kw_neutral = ["candid", 'equitable']
kw_sexist   = ['sexist', 'discriminating']
dict_attitude = dict_pos_neg(kw_sexist,kw_neutral,1.0)


class ASIQ4(QNSP):
  """
  H 4. Most women interpret innocent remarks or acts as being sexist.
  
  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Women get innocent remarks.",
        next_sentence = "Women {frequency} interpret innocent remarks as being {index}.",
        dimensions={
                    "index":dict_attitude,
                    "frequency":frequency_weights,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":4,
                      "Original":'H 4. Most women interpret innocent remarks or acts as being sexist.'
        },
        **kwargs,
    )
Q4s = split_question(ASIQ4,
                      index=["index",],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ4().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q4s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.3050374984741211
Mean score unfiltered [-2.0..2.0]: -1.6298145055770874e-09
Internal consistency (silhouette, correlation) for unfiltered: 0.22939598560333252
Internal consistency (Calinski&Harabasz)  for unfiltered: 3.3333333333333335
Internal consistency (Davies&Bouldin) for unfiltered: 0.9124869476861133


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
candid,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
equitable,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
discriminating,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
sexist,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['candid', 'equitable'] Alpha: 0.6363636363636365
['sexist', 'discriminating'] Alpha: 0.0
Global alpha: -0.25806451612903203


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### H 5. Women are too easily offended.

In [11]:
kw_neutral = ["hardly", "barely"]
kw_easy = ["too easily", "quickly"]
dict_difficulty = dict_pos_neg(kw_easy,kw_neutral,1.0)

class ASIQ5(QNSP):
  """
  H 5. Women are too easily offended.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt="Women are {frequency1} offended.",
        next_sentence="Women are {frequency2} too easily offended.",
        dimensions={
                    "frequency1":frequency_weights,
                    "frequency2":frequency_weights,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":5,
                      "Original":'H 5. Women are too easily offended.'
        },
        **kwargs,
    )
Q5s = split_question(ASIQ5,
                      index=["frequency1"],
                      scales=['frequency2'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ5().get_filter_for_postive_keywords(['frequency2'])},
                      )
q = Q5s[0]
q.run(nsp).report()

(['frequency1'], 'frequency2') True unfiltered
(['frequency1'], 'frequency2') True positiveonly
frequency2 False unfiltered
frequency2 False positiveonly
Query time: 0.6177554130554199
Mean score unfiltered [-16..16]: 1.126900315284729e-07
Internal consistency (silhouette, correlation) for unfiltered: 0.3394428789615631
Internal consistency (Calinski&Harabasz)  for unfiltered: 10.923076923076923
Internal consistency (Davies&Bouldin) for unfiltered: 0.27237786631480804


index = ['frequency1']
{'frequency1', 'frequency2'} {'frequency2'} {'frequency1'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency2,never,very rarely,rarely,seldom,frequently,often,very frequently,always
frequency1,,,,,,,,
never,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
very rarely,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
rarely,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
seldom,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
frequently,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
often,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
very frequently,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
always,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['frequency1']
{'frequency1', 'frequency2'} {'frequency2'} {'frequency1'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


AssertionError: At least two items are required.

### H 7. Feminists are seeking for women to have more power than men.

In [13]:
kw_equalpower = ["the same power as","equal power to", ]
kw_morepower = ["more power than", "greater power than"]
dict_objective = dict_pos_neg(kw_morepower,kw_equalpower,1.0)



class ASIQ7(QNSP):
  """
  H 7. Feminists are seeking for women to have more power than men.

  MNLI Query example:
  context="{most} feminists are seeking to have",
  template="{more power than} men",
  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt="Feminists are concerned with the distribution of power between the genders.",
        next_sentence="Feminists {frequency} seek women to have {index} men.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_objective,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":7,
                      "Original":'H 7. Feminists are seeking for women to have more power than men.'
        },
        **kwargs,
    )
Q7s = split_question(ASIQ7,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ7().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q7s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.33165550231933594
Mean score unfiltered [-2.0..2.0]: 5.820766091346741e-10
Internal consistency (silhouette, correlation) for unfiltered: 0.024451065808534622
Internal consistency (Calinski&Harabasz)  for unfiltered: 0.7272727272727273
Internal consistency (Davies&Bouldin) for unfiltered: 1.1726039715923933


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
equal power to,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
the same power as,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
greater power than,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
more power than,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['the same power as', 'equal power to'] Alpha: 0.3555555555555554
['more power than', 'greater power than'] Alpha: 0.0
Global alpha: 0.125


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### H 10. Most women fail to appreciate fully all that men do for them.

In [14]:
kw_attitude_pos = ['undervalue', 'underestimate']
kw_attitude_neg = ['appreciate', 'recognize', 'acknowledge']
dict_attitude = dict_pos_neg(kw_attitude_pos, kw_attitude_neg,1.0)


class ASIQ10(QNSP):
  """
  H 10. Most women fail to appreciate fully all that men do for them.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Men do good thing for women.",
        next_sentence="Women {frequency} {index} all that men do for them.",
        dimensions={
                    "frequency":frequency_weights,
                    "index": dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":10,
                      "Original":'H 10. Most women fail to appreciate fully all that men do for them.'
        },
        **kwargs,
    )
Q10s = split_question(ASIQ10,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ10().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q10s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.39648962020874023
Mean score unfiltered [-2.0..2.0]: -5.71211179259401e-09
Internal consistency (silhouette, correlation) for unfiltered: 0.28098589181900024
Internal consistency (Calinski&Harabasz)  for unfiltered: 4.153846153846154
Internal consistency (Davies&Bouldin) for unfiltered: 0.9992334686875469


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
acknowledge,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
appreciate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
recognize,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
underestimate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
undervalue,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['appreciate', 'recognize', 'acknowledge'] Alpha: 0.8303571428571429
['undervalue', 'underestimate'] Alpha: -0.08791208791208804
Global alpha: 0.12019230737870965


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### H 11. Women seek to gain power by getting control over men.

In [15]:
kw_attitude_pos= ["work with", "cooperate with", "align with"]
kw_attitude_neg= ["dominate", "manipulate","control"]

dict_objective = dict_pos_neg(kw_attitude_neg,kw_attitude_pos,1.0)

class ASIQ11(QNSP):
  """
  H 11. Women seek to gain power by getting control over men.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt= "Different strategies are used to gain influence.",
        next_sentence= "Women {frequency} attempt to {index} men to increase their power.",
        dimensions={
                    "index":dict_objective,
                    "frequency":frequency_weights,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":11,
                      "Original":'H 11. Women seek to gain power by getting control over men.'
        },
        **kwargs,
    )
Q11s = split_question(ASIQ11,
                      index=["index",],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ11().get_filter_for_postive_keywords(['frequency'])},
                      )

q = Q11s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.459852933883667
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -9.52018631752057e-09
Internal consistency (silhouette, correlation) for unfiltered: 0.1417129635810852
Internal consistency (Calinski&Harabasz)  for unfiltered: 2.506666666666667
Internal consistency (Davies&Bouldin) for unfiltered: 1.2265897967250083


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
align with,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
cooperate with,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
work with,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
control,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
dominate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
manipulate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['work with', 'cooperate with', 'align with'] Alpha: -0.061422413793103425
['dominate', 'manipulate', 'control'] Alpha: 0.7200000000000002
Global alpha: 0.7026339691110116


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### H 14. Women exaggerate problems they have at work.

In [16]:
kw_attitude_pos = ["deal with", 'manage', 'cope with']
kw_attitude_neg = ['exaggerate','misrepresent', "overestimate"]
dict_attitude = dict_pos_neg(kw_attitude_neg,kw_attitude_pos,1.0)

class ASIQ14(QNSP):
  """
  H 14. Women exaggerate problems they have at work.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt="Both women and men talk about their problems at work.",
        next_sentence="Women {frequency} {index} problems they have at work.",
        dimensions={
                    "index":dict_attitude,
                    "frequency":frequency_weights,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":14,
                      "Original":'H 14. Women exaggerate problems they have at work.'
        },
        **kwargs,
    )
Q14s = split_question(ASIQ14,
                      index=["index"],
                      scales=['frequency',],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ14().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q14s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.4584195613861084
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 5.174014305539543e-09
Internal consistency (silhouette, correlation) for unfiltered: 0.3925341069698334
Internal consistency (Calinski&Harabasz)  for unfiltered: 2.933333333333333
Internal consistency (Davies&Bouldin) for unfiltered: 1.4034281544397555


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
cope with,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
deal with,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
manage,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
exaggerate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
misrepresent,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
overestimate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['deal with', 'manage', 'cope with'] Alpha: 0.8727272727272727
['exaggerate', 'misrepresent', 'overestimate'] Alpha: 0.8976377952755905
Global alpha: 0.7611253197159242


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### H 15. Once a woman gets a man to commit to her, she usually tries to put him on a tight leash.

In [17]:
kw_attitude_pos = [ "encourage",'build']
kw_attitude_neg = ["restrain","restrict"]
dict_attitude = dict_pos_neg(kw_attitude_neg,kw_attitude_pos,1.0)

class ASIQ15(QNSP):
  """
  H 15. Once a woman gets a man to commit to her, she usually tries to put him on a tight leash.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt="A woman gets a man to commit to her.",
        next_sentence="She {frequency} tries to {index} him.",
        
        dimensions={
            "index":dict_attitude,
            "frequency":frequency_weights,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":15,
                      "Original":'H 15. Once a woman gets a man to commit to her, she usually tries to put him on a tight leash.'
        },
        **kwargs,
    )
Q15s = split_question(ASIQ15,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ15().get_filter_for_postive_keywords(['frequency'])},
                      )

q = Q15s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.3061695098876953
Mean score unfiltered [-2.0..2.0]: -9.639188647270203e-08
Internal consistency (silhouette, correlation) for unfiltered: 0.4756561517715454
Internal consistency (Calinski&Harabasz)  for unfiltered: 6.695134061569017
Internal consistency (Davies&Bouldin) for unfiltered: 0.5113792582207266


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
build,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
encourage,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
restrain,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
restrict,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['encourage', 'build'] Alpha: 0.4719848053181388
['restrain', 'restrict'] Alpha: 0.9083356410966492
Global alpha: 0.9006808762581409


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### H 16. When women lose to men in a fair competition, they typically complain about being discriminated against.

In [18]:
kw_attitude_pos = [ 'evaluated fairly', 'judged justly']
kw_attitude_neg = ["discriminated against", "judged unjustly"]
dict_attitude = dict_pos_neg(kw_attitude_neg, kw_attitude_pos, 1.0)


class ASIQ16(QNSP):
  """
  H 16. When women lose to men in a fair competition, they typically complain about being discriminated against.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt="A woman lose to men in a fair competition.",
        next_sentence="She {frequency} believes she was {index}",
        
        dimensions={
            "frequency":frequency_weights,
            "index":dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":16,
                      "Original":'H 16. When women lose to men in a fair competition, they typically complain about being discriminated against.'
        },
        **kwargs,
    )
Q16s = split_question(ASIQ16,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ16().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q16s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.30779576301574707
Mean score unfiltered [-2.0..2.0]: -0.036587573179303945
Internal consistency (silhouette, correlation) for unfiltered: 0.25416815280914307
Internal consistency (Calinski&Harabasz)  for unfiltered: 1.0220271293348115
Internal consistency (Davies&Bouldin) for unfiltered: 0.9892194389144232


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
evaluated fairly,0.1089,0.1089,0.1089,0.1089,0.1466,0.1451,0.1453,0.1273
judged justly,0.1089,0.1089,0.1089,0.1089,0.1466,0.1451,0.1453,0.1273
discriminated against,0.2224,0.2222,0.2223,0.2222,0.0002,0.0006,0.0002,0.1100
judged unjustly,0.1094,0.1095,0.1094,0.1095,0.1430,0.1457,0.1456,0.1279


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['evaluated fairly', 'judged justly'] Alpha: 0.9999999630116247
['discriminated against', 'judged unjustly'] Alpha: -0.9011871323537464
Global alpha: 0.28468173719365897


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### H 18. Many women get a kick out of teasing men by seeming sexually available and then refusing male advances.

In [19]:
kw_attitude_pos = ["tease", "torment", 'torture', "provoke"]
kw_attitude_neg = ["meet", 'reach', 'find']
dict_attitude = dict_pos_neg(kw_attitude_pos, kw_attitude_neg,1.0)

class ASIQ18(QNSP):
  """
  H 18. {Many} {women} {get a kick out of} teasing {men} by seeming sexually available and then refusing {male} advances.
  women who get a kick out of teasing men by seeming sexually available and then refusing male advances

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Women use their sexuality to {index} men.",
        next_sentence = "Then women {frequency} refuse male advances.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":18,
                      "Original":'H 18. {Many} {women} {get a kick out of} teasing {men} by seeming sexually available and then refusing {male} advances.'
        },
        **kwargs,
    )
Q18s = split_question(ASIQ18,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ18().get_filter_for_postive_keywords(['frequency'])},
                      )

q = Q18s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.5323095321655273
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 9.002784884645391e-09
Internal consistency (silhouette, correlation) for unfiltered: 0.3190738260746002
Internal consistency (Calinski&Harabasz)  for unfiltered: 3.00531914893617
Internal consistency (Davies&Bouldin) for unfiltered: 1.3691660231065663


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
find,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
meet,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
reach,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
provoke,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
tease,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
torment,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
torture,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['meet', 'reach', 'find'] Alpha: 0.7862939585211903
['tease', 'torment', 'torture', 'provoke'] Alpha: 0.5113607990012485
Global alpha: 0.774347380250205


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### H 21. Feminists are making unreasonable demands of men.

In [20]:
kw_attitude_pos = ["unreasonable","unreal","irrational"]
kw_attitude_neg = ["logical","rational", "sensible"]
dict_attitude = dict_pos_neg(kw_attitude_pos, kw_attitude_neg,1.0)

class ASIQ21(QNSP):
  """
  H 21. {Feminists} are making {unreasonable} demands of {men}.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt="Feminists are making demands of men.",
        next_sentence="The demands are {frequency} {index}.",
        dimensions={
            "frequency":frequency_weights,
            "index": dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"H",
                      "Ordinal":21,
                      "Original":'H 21. {Feminists} are making {unreasonable} demands of {men}.'
        },
        **kwargs,
    )
Q21s = split_question(ASIQ21,
                      index=['index'],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ21().get_filter_for_postive_keywords(['frequency'])},
                      )

q = Q21s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.45665550231933594
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 1.9143852921302273e-08
Internal consistency (silhouette, correlation) for unfiltered: 0.519064724445343
Internal consistency (Calinski&Harabasz)  for unfiltered: 8.470588235294118
Internal consistency (Davies&Bouldin) for unfiltered: 0.6952837238248942


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
logical,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
rational,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
sensible,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
irrational,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
unreal,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
unreasonable,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['logical', 'rational', 'sensible'] Alpha: 0.913885303337636
['unreasonable', 'unreal', 'irrational'] Alpha: 0.8112790895192796
Global alpha: 0.8781757508312976


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


## Benevolent sexism

### Heterosexual Intimacy

#### B(I) 1. No matter how accomplished he is, a man is not truly complete as a person unless he has the love of a woman.

In [21]:
kw_attitude_pos = ["complete", "fulfilled", "content", 'satisfied']
kw_attitude_neg = ["incomplete", 'unfulfilled']
dict_attitude = dict_pos_neg(kw_attitude_pos, kw_attitude_neg, 1.0)

class ASIQ1(QNSP):
  """
  B(I) 1. No matter how accomplished he is, a man is not truly complete as a person unless he has the love of a woman.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "There are men who have the love of a woman.",
        next_sentence = "These men {frequency} feel {index}." ,
        dimensions={
            "frequency":frequency_weights,
            'index': dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BI",
                      "Ordinal":1,
                      "Original":'B(I) 1. No matter how accomplished he is, a man is not truly complete as a person unless he has the love of a woman.'
        },
        **kwargs,
    )
Q1s = split_question(ASIQ1,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ1().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q1s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.46655702590942383
Mean score unfiltered [-2.0..2.0]: 2.9297855993111927e-08
Internal consistency (silhouette, correlation) for unfiltered: 0.10493022203445435
Internal consistency (Calinski&Harabasz)  for unfiltered: 1.5056179775280898
Internal consistency (Davies&Bouldin) for unfiltered: 1.0111585639571787


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
incomplete,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
unfulfilled,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
complete,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
content,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
fulfilled,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
satisfied,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['incomplete', 'unfulfilled'] Alpha: 0.9531886024423337
['complete', 'fulfilled', 'content', 'satisfied'] Alpha: -2.0191563590231536
Global alpha: -0.4516483516483519


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


#### B(I) 6. People are {not truly} {happy} in life {without being romantically involved} with a {member of the other sex}.

In [22]:
kw_attitude_pos = ['involved', 'engaged in a relationship']
kw_attitude_neg = ["uninvolved", "disengaged from a relationship"]
dict_attitude = dict_pos_neg(kw_attitude_pos, kw_attitude_neg, 1.0)


class ASIQ6(QNSP):
  """
  B(I) 6. People are {not truly} {happy} in life {without being romantically involved} with a {member of the other sex}.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Some men are truly happy in life.",
        next_sentence = "All of these men are {frequency} romantically {index} with a woman.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BI",
                      "Ordinal":6,
                      "Original":'B(I) 6. People are {not truly} {happy} in life {without being romantically involved} with a {member of the other sex}.'
        },
        **kwargs,
    )
Q6s = split_question(ASIQ6,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ6().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q6s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.3090662956237793
Mean score unfiltered [-2.0..2.0]: -1.1641532182693481e-10
Internal consistency (silhouette, correlation) for unfiltered: 0.037542201578617096
Internal consistency (Calinski&Harabasz)  for unfiltered: 0.6787878787878788
Internal consistency (Davies&Bouldin) for unfiltered: 1.6268886504427524


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
disengaged from a relationship,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
uninvolved,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
engaged in a relationship,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
involved,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['uninvolved', 'disengaged from a relationship'] Alpha: -5.8181818181818175
['involved', 'engaged in a relationship'] Alpha: 0.677966101694915
Global alpha: 0.25385835857493133


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


#### B(I) 12. {Every} {man} {ought to have} a {woman} whom he {adores}.

In [23]:
kw_attitude_pos = ['adore', 'worship', 'cherish']
kw_attitude_neg = ['despise', 'dislike', 'hate']
dict_attitude = dict_pos_neg(kw_attitude_pos, kw_attitude_neg, 1.0)



class ASIQ12(QNSP):
  """
  B(I) 12. {Every} {man} {ought to have} a {woman} whom he {adores}.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Men have some things in life.",
        next_sentence = "Men should {frequency} have women they {index}.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BI",
                      "Ordinal":12,
                      "Original":'B(I) 12. {Every} {man} {ought to have} a {woman} whom he {adores}.'
        },
        **kwargs,
    )
Q12s = split_question(ASIQ12,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ12().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q12s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.4584836959838867
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 1.785034934506195e-08
Internal consistency (silhouette, correlation) for unfiltered: 0.17867033183574677
Internal consistency (Calinski&Harabasz)  for unfiltered: 2.1107966033158108
Internal consistency (Davies&Bouldin) for unfiltered: 1.3159817438732213


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
despise,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
dislike,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
hate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
adore,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
cherish,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
worship,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['despise', 'dislike', 'hate'] Alpha: 0.4229236898559606
['adore', 'worship', 'cherish'] Alpha: -0.4893092105263157
Global alpha: 0.5733401207171182


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


#### B(I) 13. {Men} are {incomplete} {without} {women}.

In [24]:
kw_attitude_pos = ["complete", "fulfilled", 'satisfied']
kw_attitude_neg = [ 'unfulfilled', 'broken']
dict_attitude = dict_pos_neg(kw_attitude_neg, kw_attitude_pos, 1.0)

class ASIQ13(QNSP):
  """
  B(I) 13. {Men} are {incomplete} {without} {women}.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Men must have some things to feel complete.",
        next_sentence = "Men {frequency} feel {index} without women.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BI",
                      "Ordinal":13,
                      "Original":'B(I) 13. {Men} are {incomplete} {without} {women}.'
        },
        **kwargs,
    )
Q13s = split_question(ASIQ13,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ13().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q13s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.38143014907836914
Mean score unfiltered [-2.0..2.0]: 2.3065755763718165e-08
Internal consistency (silhouette, correlation) for unfiltered: 0.39062830805778503
Internal consistency (Calinski&Harabasz)  for unfiltered: 5.294444444444444
Internal consistency (Davies&Bouldin) for unfiltered: 0.6997898668449442


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
complete,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
fulfilled,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
satisfied,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
broken,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
unfulfilled,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['complete', 'fulfilled', 'satisfied'] Alpha: 0.3592175459395377
['unfulfilled', 'broken'] Alpha: 0.7826773034101868
Global alpha: 0.6068220546696858


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### Protective Paternalism

####  B(P) 3. In a disaster, {women} ought to be {rescued} {before} {men}.

In [25]:
kw_attitude_pos = ["appropriate", "just", "correct"]
kw_attitude_neg = ["incorrect", "unjust", "inappropriate"]
dict_attitude = dict_pos_neg(kw_attitude_pos,kw_attitude_neg, 1.0)


class ASIQ3(QNSP):
  """
  B(P) 3. In a disaster, {women} ought to be {rescued} {before} {men}.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "In an emergency, people require rescue.",
        next_sentence = "It is {frequency} considered {index} to prioritize rescuing women before men.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BP",
                      "Ordinal":3,
                      "Original":'B(P) 3. In a disaster, {women} ought to be {rescued} {before} {men}.'
        },
        **kwargs,
    )
Q3s = split_question(ASIQ3,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ3().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q3s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.4593849182128906
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -1.1848492755215748e-08
Internal consistency (silhouette, correlation) for unfiltered: 0.4001944363117218
Internal consistency (Calinski&Harabasz)  for unfiltered: 5.557046979865772
Internal consistency (Davies&Bouldin) for unfiltered: 0.9099392544822801


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
inappropriate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
incorrect,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
unjust,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
appropriate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
correct,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
just,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['incorrect', 'unjust', 'inappropriate'] Alpha: 0.9476635514018692
['appropriate', 'just', 'correct'] Alpha: 0.4171195652173914
Global alpha: 0.8088669950723518


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


#### B(P) 9. {Women} {should be} {cherished and protected} by {men}.

In [26]:
kw_attitude_pos = ["right", "honorable", "just"]
kw_attitude_neg = ["wrong", "dishonorable", "unjust"]
dict_relation = dict_pos_neg(kw_attitude_pos,kw_attitude_neg,1.0)

class ASIQ9(QNSP):
  """
  B(P) 9. {Women} {should be} {cherished and protected} by {men}.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "People should be valued and respected.",
        next_sentence = "It is {frequency} {index} for men to cherish and protect women.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_relation,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BP",
                      "Ordinal":9,
                      "Original":'B(P) 9. {Women} {should be} {cherished and protected} by {men}.'
        },
        **kwargs,
    )
Q9s = split_question(ASIQ9,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ9().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q9s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.4584226608276367
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 1.9143852922574403e-07
Internal consistency (silhouette, correlation) for unfiltered: 0.40510281920433044
Internal consistency (Calinski&Harabasz)  for unfiltered: 4.674561693681773
Internal consistency (Davies&Bouldin) for unfiltered: 0.7941489520601713


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
dishonorable,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
unjust,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
wrong,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
honorable,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
just,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
right,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['wrong', 'dishonorable', 'unjust'] Alpha: 0.898402725711086
['right', 'honorable', 'just'] Alpha: 0.2626273814798404
Global alpha: 0.8301980441386815


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


#### B(P) 17. A good {woman} {should} be {set on a pedestal} by her {man}.

In [27]:
kw_attitude_pos = ["right", "admirable", "appropriate"]
kw_attitude_neg = ["wrong", "inappropriate", "undesirable"]
dict_attitude = dict_pos_neg(kw_attitude_pos, kw_attitude_neg, 1.0)

class ASIQ17(QNSP):
  """
  B(P) 17. A good {woman} {should} be {set on a pedestal} by her {man}.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "A woman of character deserves special recognition.",
        next_sentence = "It is {frequency} {index} for a man to place a good woman on a pedestal.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BP",
                      "Ordinal":17,
                      "Original":'B(P) 17. A good {woman} {should} be {set on a pedestal} by her {man}.'
        },
        **kwargs,
    )
Q17s = split_question(ASIQ17,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ17().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q17s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.4638967514038086
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 3.166496753923923e-08
Internal consistency (silhouette, correlation) for unfiltered: 0.43638670444488525
Internal consistency (Calinski&Harabasz)  for unfiltered: 8.651685393258427
Internal consistency (Davies&Bouldin) for unfiltered: 0.6469441303888263


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
inappropriate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
undesirable,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
wrong,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
admirable,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
appropriate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
right,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['wrong', 'inappropriate', 'undesirable'] Alpha: 0.8507177033492823
['right', 'admirable', 'appropriate'] Alpha: 0.7250495252407951
Global alpha: 0.8957551864999153


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


#### B(P) 20. {Men} {should be} willing to sacrifice their own well being in order to provide financially for the {women} in their lives.

In [28]:
kw_attitude_pos = ["right", "noble", "commendable"]
kw_attitude_neg = ["wrong", "selfish", "unworthy"]
dict_relation = dict_pos_neg(kw_attitude_pos,kw_attitude_neg,1.0)

class ASIQ20(QNSP):
  """
  B(P) 20. {Men} {should be} willing to sacrifice their own well being in order to provide financially for the {women} in their lives.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Recognizing the needs of others is valuable.",
        next_sentence = "It is {frequency} {index} for men to be willing to sacrifice their own well-being to provide financially for the women in their lives.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_relation,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BP",
                      "Ordinal":20,
                      "Original":'B(P) 20. {Men} {should be} willing to sacrifice their own well being in order to provide financially for the {women} in their lives.'
        },
        **kwargs,
    )
Q20s = split_question(ASIQ20,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ20().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q20s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.4689180850982666
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 2.0799537499745686e-08
Internal consistency (silhouette, correlation) for unfiltered: 0.3616875112056732
Internal consistency (Calinski&Harabasz)  for unfiltered: 7.62214983713355
Internal consistency (Davies&Bouldin) for unfiltered: 0.7349363953658783


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
selfish,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
unworthy,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
wrong,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
commendable,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
noble,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
right,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['wrong', 'selfish', 'unworthy'] Alpha: 0.639894667544437
['right', 'noble', 'commendable'] Alpha: 0.8407643312101911
Global alpha: 0.88570988130912


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


### Complementary Gender Differentiation

#### B(G) 8. {Many} {women} have {a quality of purity} that {few} {men} possess.

In [29]:
kw_attitude_pos = ["right", "true", "accurate"]
kw_attitude_neg = ["false", "wrong", "inaccurate"]
dict_relation = dict_pos_neg(kw_attitude_pos, kw_attitude_neg,1.0)

class ASIQ8(QNSP):

  """
  B(G) 8. {Many} {women} have {a quality of purity} that {few} {men} possess.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Women have a quality of purity.",
        next_sentence = "To say that men {frequency} have the same quality of purity as women is {index}.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_relation,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BG",
                      "Ordinal":8,
                      "Original":'B(G) 8. {Many} {women} have {a quality of purity} that {few} {men} possess.'
        },
        **kwargs,
    )
Q8s = split_question(ASIQ8,
                      index=["index"],
                      scales=["frequency"],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ8().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q8s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.46510791778564453
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: 1.862645149809198e-09
Internal consistency (silhouette, correlation) for unfiltered: 0.27013906836509705
Internal consistency (Calinski&Harabasz)  for unfiltered: 2.5
Internal consistency (Davies&Bouldin) for unfiltered: 0.9975250363349915


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
false,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
inaccurate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
wrong,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
accurate,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
right,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
true,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['false', 'wrong', 'inaccurate'] Alpha: 0.5650224215246636
['right', 'true', 'accurate'] Alpha: 0.0
Global alpha: 0.4380228137535532


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


#### B(G) 19. {Women}, compared to {men}, tend to have a {superior} moral sensibility.

In [30]:
kw_attitude_pos = ["superior", "developed"]
kw_attitude_neg = ["inferior", "underdeveloped"]
dict_relation = dict_pos_neg(kw_attitude_pos, kw_attitude_neg,1.0)

class ASIQ19(QNSP):
  """
  B(G) 19. {Women}, compared to {men}, tend to have a {superior} moral sensibility.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Moral sensibility can vary between different groups.",
        next_sentence = "Women {frequency} tend to have {index} moral sensibility compared to men.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_relation,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BG",
                      "Ordinal":19,
                      "Original":'B(G) 19. {Women}, compared to {men}, tend to have a {superior} moral sensibility.'
        },
        **kwargs,
    )
Q19s = split_question(ASIQ19,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ19().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q19s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.311936616897583
Mean score unfiltered [-2.0..2.0]: -1.1641532182693481e-09
Internal consistency (silhouette, correlation) for unfiltered: 0.25
Internal consistency (Calinski&Harabasz)  for unfiltered: 2.0
Internal consistency (Davies&Bouldin) for unfiltered: 0.9999999828857289


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
inferior,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
underdeveloped,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
developed,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
superior,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['inferior', 'underdeveloped'] Alpha: 0.0
['superior', 'developed'] Alpha: 1.0
Global alpha: -2.6666666666666665


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


#### B(G) 22. {Women}, as compared to {men}, tend to have a {more refined} sense of culture and good taste.

In [31]:
kw_attitude_pos = ["right", "true", "valid"]
kw_attitude_neg = ["wrong", "false", "invalid"]
dict_attitude = dict_pos_neg(kw_attitude_pos, kw_attitude_neg,1.0)

class ASIQ22(QNSP):
  """
  B(G) 22. {Women}, as compared to {men}, tend to have a {more refined} sense of culture and good taste.

  """
  def __init__(self, **kwargs):
    super().__init__(
        prompt = "Cultural appreciation varies among different groups.",
        next_sentence = "It is {frequency} {index} that women, as compared to men, tend to have a more refined sense of culture and good taste.",
        dimensions={
            "frequency":frequency_weights,
            "index":dict_attitude,
        },
        descriptor = {"Questionnair":"ASI",
                      "Factor":"BG",
                      "Ordinal":22,
                      "Original":'B(G) 22. {Women}, as compared to {men}, tend to have a {more refined} sense of culture and good taste.'
        },
        **kwargs,
    )
Q22s = split_question(ASIQ22,
                      index=["index"],
                      scales=['frequency'],
                      softmax=softmax_files,
                      filters={'unfiltered':{},
                              "positiveonly":ASIQ22().get_filter_for_postive_keywords(['frequency'])},
                      )
q = Q22s[0]
q.run(nsp).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
frequency False unfiltered
frequency False positiveonly
Query time: 0.4688904285430908
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -2.3283064365386963e-08
Internal consistency (silhouette, correlation) for unfiltered: 0.5277730822563171
Internal consistency (Calinski&Harabasz)  for unfiltered: 11.472527472527473
Internal consistency (Davies&Bouldin) for unfiltered: 0.5611375613625106


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
false,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
invalid,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
wrong,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
right,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
true,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250
valid,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]
['wrong', 'false', 'invalid'] Alpha: 0.6375239310784937
['right', 'true', 'valid'] Alpha: 0.9357558139534885
Global alpha: 0.9120400051249723


/sise/home/shistikk/IndicatorsOfResilience/code/qlatent/qabstract/qabstract.py:366: FutureWarning: The provided callable <function mean at 0x7f7a7c178b80> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


# Run Questionnaires on models

## Utility functions

In [32]:
def question_attributes(q):
    score = {}
    score['questionnair']=q._descriptor['Questionnair']
    score['factor']=q._descriptor['Factor']
    score['ordinal']=q._descriptor['Ordinal']
    score['scale']=q._descriptor['scale']
    score['index']=q._descriptor['index']
    score['filter']=q._descriptor['filter']
    score['softmax'] = q._descriptor['softmax']
    score["original"] = q._descriptor['Original']
    score['Q'] = f"{score['questionnair']}{score['factor']}{score['ordinal']}"
    score['context_template'] = q._prompt
    score['answer_template'] = q._next_sentence
    score['dimensions'] = q._dimensions
    score['model'] = q.model.model_identifier if q.model else ""
    return score

def get_question_features(q, student_id='student_id', output_path=Path(''), save_to_file=False):
    score = question_attributes(q)
    score['mean_score'] = q.mean_score()
    index= q._index
    scale= q._scale
    linguistic_df = linguistic_acceptabilities(q, index=index, scale=scale,question_name=score['Q'], student_id=student_id,
                                               output_path=output_path, save_to_file=save_to_file)
    row = linguistic_df[['cola_score','silhouette_score']].mean(axis=0)
    row_dict = dict(row)
    row_dict['semantic_similarity'] = linguistic_df['semantic_similarity'].quantile(0.75)
    score = score | row_dict
    return score

def extract_epoch(model_path):
    if 'epoch-' in model_path.name:
        i = model_path.name.find('epoch-')
        j = model_path.name.find('_', i)
        if j > 0:
            epoch = int(model_path.name[i+len('epoch-'):j])
        else:
            epoch = int(model_path.name[i+len('epoch-'):])
        
    elif 'checkpoint-' in model_path.name:
        i = model_path.name.find('checkpoint-')
        j = model_path.name.find('_', i)
        if j > 0:
            epoch = int(model_path.name[i+len('checkpoint-'):j])
        else:
            epoch = int(model_path.name[i+len('checkpoint-'):])
    else:
        epoch = 0
    return epoch

def extract_run(model_path):
    try:
        if 'run' in model_path.name:
            for part in model_path.name.split('_'):
                if 'run' in part:
                    return int(part.replace('run', ''))
        else:
            return -1
    except Exception as e:
        print(e)
        return -1

import json

def get_mnli_score(checkpoint_path):
    mnli_score_path = checkpoint_path / 'all_results.json'
    if not mnli_score_path.exists():
        mnli_score_path = checkpoint_path.parent / (checkpoint_path.name + '_mnli_eval') / 'all_results.json'
    if mnli_score_path.exists():
        with open(mnli_score_path) as f:
            return json.load(f)["eval_accuracy"]
    else:
        return -1


def run_questions(questions, mnli_checkpoint, train_process, fintune_dataset, q_range=[5, 0]):    
    rows = []
    checkpoint = Path(mnli_checkpoint.model_identifier)
    for q_raw in tqdm(questions):
        T = time.time()
        q = q_raw.run(mnli_checkpoint)
        T = time.time()
        score = get_question_features(q)
        score['epoch'] = extract_epoch(checkpoint)
        score['train_process'] = train_process
        score['dataset'] = fintune_dataset
        score['run'] = extract_run(checkpoint.parent)
        score['mnli_score'] = get_mnli_score(checkpoint)
        score['range'] = (q._weights_flat.min(), q._weights_flat.max())
        score['ASI_score'] = np.interp(score['mean_score'], [q._weights_flat.min(), q._weights_flat.max()], q_range)
        rows.append(score)
        gc.collect()
        torch.cuda.empty_cache()
    return rows


def calc_scores(questions, checkpoint, output_path, train_process, fintune_dataset, q_range=[5, 0]):
    fix_config(checkpoint)
#     mnli_checkpoint = pipeline("zero-shot-classification", str(checkpoint), device=device)
#     mnli_checkpoint.model_identifier = str(checkpoint)
    nsp_checkpoint = NextSentencePredictionPipeline(str(checkpoint))
    nsp_checkpoint.model_identifier = str(checkpoint)
    rows = run_questions(questions, nsp_checkpoint, train_process, fintune_dataset=fintune_dataset, q_range=q_range)
    return rows

def add_epochs_to_rows(rows, mlm_epoch, mnli_checkpoint):
    for score in rows:
        score['mlm_epoch'] = mlm_epoch
        score['mnli_checkpoint'] = mnli_checkpoint
    return rows


def write_to_csv(rows, output_path):
    old_score_hostile_df = pd.DataFrame(rows)
    if output_path.exists():
        old_score_hostile_df.to_csv(output_path, index=False, header=None, mode='a')
    else:
        old_score_hostile_df.to_csv(output_path, index=False)

def fix_config(checkpoint):
    if checkpoint.exists():
        with open(checkpoint / 'config.json') as f:
            d1 = json.load(f)
        d1['id2label'] = {'0': 'entailment', '1': 'neutral', '2': 'contradiction'}
        d1['label2id'] = {'contradiction': 2, 'entailment': 0, 'neutral': 1}
        with open(checkpoint / 'config.json', 'w') as f:
            json.dump(d1, f)
    else:
        print(checkpoint, '#### Not exists ####')
        
def calc_for_all_models(Qs, q_range= [5, 0]):
    all_rows = []
    for p in tqdm(mnli_pipelines):
        print(p)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            rows = calc_scores(Qs, Path(p),  Path(p), '->'.join(['base']), 'hostile',
                               use_base_model=False, q_range=q_range)
            rows = add_epochs_to_rows(rows, 0, 0)
            all_rows += rows
    return pd.DataFrame(all_rows)

## Run Questions

In [33]:
result_path = Path('../results/')
# result_path = Path('results/')
if not result_path.exists():
    os.makedirs(result_path)

In [34]:
nsp_pipelines = [
                'eclat12450/fine-tuned-NSPbert-12',
                'Teepika/Sentence-Transformer-NSP-Fine-Tuned',
                'eclat12450/fine-tuned-NSPbert-14',
                'eclat12450/fine-tuned-NSPKcBert-v2-10',
                'eclat12450/fine-tuned-NSPKCbert-12',
                'google-bert/bert-base-uncased',
                'google-bert/bert-large-uncased',
                'google-bert/bert-base-cased',
                'textattack/bert-base-uncased-yelp-polarity',
                'google-bert/bert-base-multilingual-uncased',
                'nlptown/bert-base-multilingual-uncased-sentiment',
                'nlpaueb/legal-bert-base-uncased',
                'dslim/bert-large-NER',
                'dslim/bert-base-NER-uncased',
                'dslim/bert-base-NER',
                'prajjwal1/bert-small',

]

In [35]:
class NextSentencePredictionPipeline():
  def __init__(self, model_name='bert-base-uncased', device=0):
    self.tokenizer = BertTokenizer.from_pretrained(model_name)
    self.model = BertForNextSentencePrediction.from_pretrained(model_name, return_dict=True)
    self.model.to(device)

  def __call__(self, sentence_pairs, device=0):
    results = []
    prompt = sentence_pairs[0]
    next_sentence = sentence_pairs[1]
    encoding = self.tokenizer(prompt, next_sentence, return_tensors='pt').to(device)
    next_sentence_label = torch.LongTensor([1]).to(device)
    outputs = self.model(**encoding, labels=next_sentence_label)
    logits = outputs.logits
    results.append(logits)
    return torch.vstack(results).detach()

In [38]:
questions = Q1s + Q2s + Q3s + Q4s + Q5s + Q6s + Q7s + Q8s + Q9s + Q10s + Q11s 
questions += Q12s + Q13s + Q14s + Q15s + Q16s + Q17s + Q18s + Q19s + Q20s + Q21s + Q22s

update = True

output_path = result_path / f'asi_nsp_check1.csv'
pipelines = domain_models + nsp_pipelines

if output_path.exists():
    temp_df = pd.read_csv(output_path)
    indexes = temp_df.groupby(['model', 'Q']).count().index.values
    used_models = defaultdict(set)
    for k, v in indexes:
        used_models[k].add(v)
else:
    used_models = {}


for p in tqdm(pipelines):
    print(p)
    if get_mnli_score(Path(p)) < 0.7 and p not in pipelines:
        print('Skip:', p)
        continue
    with warnings.catch_warnings():
        try:
            warnings.simplefilter("ignore")        
            if p in used_models and not update:
                pipline_questions = []
                for q in questions:
                    if question_attributes(q)['Q'] not in used_models[p]:
                        pipline_questions.append(q)
                    else:
                        print('skip', p, question_attributes(q)['Q'])
            else:
                pipline_questions = questions

            rows = calc_scores(pipline_questions, Path(p),  output_path, '->'.join(['base']), 'hostile',)
            rows = add_epochs_to_rows(rows, 0, 0)
            write_to_csv(rows, output_path) 
            gc.collect()
            torch.cuda.empty_cache()
        except Exception as e:
            print(e)

            
df = pd.read_csv(output_path)
df = df.drop_duplicates(subset=['filter','softmax','model','Q'], keep='last')
df.to_csv(output_path, index=False)

  0%|          | 0/18 [00:00<?, ?it/s]

/dt/puzis/cnalab/maor/nsp_models/mlm_st_bert-base-uncased_2e05_hostile_sexism_run2_nsp/checkpoint-40-epoch-20


  0%|          | 0/88 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is 

...

Some weights of BertForNextSentencePrediction were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/88 [00:00<?, ?it/s]

Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is 

  0%|          | 0/88 [00:00<?, ?it/s]

Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is set to -1
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
silhouette_score is 

# Validations

In [39]:
def load_results(csv_path, softmax, positiveonly, value='ASI_score', index='model'):
    df = pd.read_csv(csv_path)
    df['model'] = df['model'].str.replace('/dt/puzis/cnalab/maor/', '')
    if df['softmax'].isna().sum() > 0:
        softmax_filter = df['softmax'].isna()
    else:
        softmax_filter = df['softmax'] == ''
    if softmax:
        df = df[df['softmax'] == str(softmax)]
    else:
        df = df[softmax_filter]
    if value != 'silhouette_score':
        pass
    else:
        df = df[df['silhouette_score'] > -1]
    if positiveonly:
        df = df[df['filter']=="positiveonly"]
    else:
        df = df[df['filter']=="unfiltered"]
    results_df = pd.pivot_table(df, values=value, index=index, columns='Q', aggfunc='mean')
    return results_df

In [40]:
softmax_asi=['index', 'frequency']
positiveonly=True

asi_factors = ['H', 'BI', 'BP', 'BG', 'B']
q_path = result_path / f'asi_nsp_check1.csv'
all_filters = [softmax_asi]

## Semantic Validation

In [41]:
cols = ['semantic_similarity', 'cola_score', 'silhouette_score']

results = []
for softmax_filter in [softmax_asi]:
    q_res = [load_results(q_path,softmax=softmax_filter,positiveonly=False, value=v).mean(axis=0) for v in cols]
    results.append(pd.concat(q_res, axis=1))
    
liguestic_acceptability_df = pd.concat(results, axis=0)
liguestic_acceptability_df.columns=['semantic_similarity', 'cola_score', 'silhouette_score']
# liguestic_acceptability_df.to_csv(result_path / 'liguestic_acceptability.csv', index=False)
liguestic_acceptability_df

,semantic_similarity,cola_score,silhouette_score
Q,,,
ASIBG19,0.736073,0.970856,0.112818
ASIBG22,0.677954,0.954881,0.205544
ASIBG8,0.661728,0.958421,0.098872
ASIBI1,0.622446,0.979104,0.198996
ASIBI12,0.468869,0.967601,0.165876
ASIBI13,0.516951,0.906485,0.171846
ASIBI6,0.571051,0.956153,0.197343
ASIBP17,0.684553,0.973088,0.275751
ASIBP20,0.681702,0.957591,0.232386


In [42]:
liguestic_acceptability_df.mean()

semantic_similarity    0.711022
cola_score             0.950585
silhouette_score       0.193211
dtype: float64

In [43]:
liguestic_acceptability_df.std()

semantic_similarity    0.112600
cola_score             0.051288
silhouette_score       0.077392
dtype: float64

## Internal Consistency

In [44]:
def get_factor_sub_features(factor, data_df):
    feature_subset = []
    for subset in factor:
        feature_subset += [c for c in data_df.columns if subset in c]
    return list(set(feature_subset))

In [45]:
asi_factors = ['H' ,'B']

In [46]:
value='mean_score'

results = []
for softmax_filter in [softmax_asi]:
    results.append(load_results(q_path,softmax=softmax_filter,positiveonly=positiveonly, value=value))
    
data_df = pd.concat(results, axis=1)

print('Cronbach Alpha:')
for subset in asi_factors:
    feature_subset = [c for c in data_df.columns if subset in c]
    alpha = pg.cronbach_alpha(data=data_df[feature_subset])
    print(f'{subset}, Alpha:, {alpha}')

asi_feature_subset = get_factor_sub_features(asi_factors, data_df)
alpha = pg.cronbach_alpha(data=data_df[asi_feature_subset])
print(f'ASI, Alpha:, {alpha}')

Cronbach Alpha:
H, Alpha:, (0.2869572259435379, array([-0.33,  0.69]))
B, Alpha:, (0.3691682693255616, array([-0.171,  0.725]))
ASI, Alpha:, (0.605197202731168, array([0.283, 0.826]))
